In [493]:
import pandas as pd
import numpy as np 
import datetime as dt
import re
import os
import geojson
import seaborn as sns
#from shapely.geometry import shape, GeometryCollection
#from shapely.geometry import Polygon, MultiPolygon

#import matplotlib.pyplot as plt
#import scipy.stats as stats
#from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import SGDRegressor, SGDClassifier
from  math import radians
#from sklearn.pipeline import Pipeline
#from sklearn.cluster import MiniBatchKMeans, KMeans
#from sklearn.metrics import mean_squared_error, mean_absolute_error




In [299]:
raw = pd.read_parquet(f'D:/TOPIC_RELATED VOCABULARY IELTS/Finland/aalto/cemsschool/Other Courses/programming course for DS/project//trip_data/06.parquet', engine='pyarrow')


In [7]:
raw.describe()

,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,499043.000000,499043.000000,499043.000000,499043.000000,499043.000000,499043.000000,499043.000000,499043.00000,499043.000000,499043.000000,499043.000000,499043.000000,499043.000000,499043.000000
mean,1.356148,2.864602,1.047214,160.197672,155.456091,1.373327,11.844105,1.12473,0.491592,1.772877,0.221110,0.296680,17.027168,2.103727
std,1.016665,32.119857,1.095790,68.832677,73.466849,0.531616,11.449169,1.28302,0.081359,2.696115,1.368441,0.043455,13.431950,0.939306
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-216.000000,-4.50000,-0.500000,-36.300000,-28.750000,-0.300000,-216.300000,-2.500000
25%,1.000000,1.000000,1.000000,107.000000,90.000000,1.000000,6.000000,0.00000,0.500000,0.000000,0.000000,0.300000,10.300000,2.500000
50%,1.000000,1.710000,1.000000,161.000000,153.000000,1.000000,8.500000,0.50000,0.500000,1.500000,0.000000,0.300000,13.560000,2.500000
75%,1.000000,3.140000,1.000000,234.000000,234.000000,2.000000,13.500000,2.50000,0.500000,2.650000,0.000000,0.300000,18.950000,2.500000
max,9.000000,22543.990000,99.000000,265.000000,265.000000,5.000000,941.500000,87.56000,3.300000,422.680000,114.750000,0.300000,1141.100000,2.500000


In [300]:
raw['duration']=(raw['tpep_dropoff_datetime']-raw['tpep_pickup_datetime']).dt.total_seconds()

In [9]:
raw.isnull().any()

tpep_pickup_datetime     False
tpep_dropoff_datetime    False
passenger_count          False
trip_distance            False
RatecodeID               False
PULocationID             False
DOLocationID             False
payment_type             False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge     False
duration                 False
dtype: bool

### HANDLING MEANINGLESS DATA

In [301]:
# LOCATION CONDITIONS
idxloc=raw['PULocationID']<=263
idx1loc=raw['DOLocationID']<=263

In [302]:
lofiltered= raw[idxloc & idx1loc]

In [303]:
lofiltered.describe()

,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
count,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,491961.000000,4.919610e+05
mean,1.357301,2.770129,1.031734,159.254644,154.296231,1.371861,11.634444,1.131160,0.492387,1.755171,0.201795,0.296825,16.799067,2.122150,7.463507e+02
std,1.018919,3.371831,0.785021,68.407615,72.975357,0.530393,10.347703,1.284881,0.078596,2.538813,1.235586,0.042769,12.171904,0.922274,4.556950e+04
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-216.000000,-4.500000,-0.500000,-36.300000,-28.750000,-0.300000,-216.300000,-2.500000,-3.187387e+07
25%,1.000000,1.000000,1.000000,107.000000,90.000000,1.000000,6.000000,0.000000,0.500000,0.000000,0.000000,0.300000,10.300000,2.500000,3.190000e+02
50%,1.000000,1.710000,1.000000,152.000000,151.000000,1.000000,8.500000,0.500000,0.500000,1.500000,0.000000,0.300000,13.550000,2.500000,5.280000e+02
75%,1.000000,3.110000,1.000000,234.000000,233.000000,2.000000,13.000000,2.500000,0.500000,2.650000,0.000000,0.300000,18.800000,2.500000,8.550000e+02
max,9.000000,240.700000,99.000000,263.000000,263.000000,4.000000,631.000000,87.560000,0.500000,422.680000,95.800000,0.300000,944.050000,2.500000,2.698540e+05


In [304]:
#DEALING WITH FEATURES THAT ARE ONLY MEANINGFUL WHEN IT IS >=0
    #APPROACH1
c03=lofiltered['extra']>=0
c04=lofiltered['mta_tax']>=0
c05=lofiltered['tip_amount']>=0
c06=lofiltered['tolls_amount']>=0
c07=lofiltered['improvement_surcharge']>=0
c08=lofiltered['total_amount']>=0
c09=lofiltered['congestion_surcharge']>=0
lonilfiltered0= lofiltered[ c03 & c04 & c05 & c06 & c07 & c08 & c09]
    #APPROACH 2 
#lonilfiltered=np.abs(lofiltered.iloc[:,2::])
#lonilfiltered= pd.concat((lofiltered.iloc[:,0:2],lonilfiltered),axis=1)



In [305]:
#DEALING WITH FEATURES THAT ARE ONLY MEANINGFUL WHEN IT IS >0
    #APPROACH 1 
#DURATIOIN CONDITION
c01=  lonilfiltered0['duration']>0
# PASSENGER COUNT CONDITION 
c02= lonilfiltered0['passenger_count']>0

lonilfiltered= lonilfiltered0[c01 & c02]

In [306]:
# DEALING WITH FEATURES trip distance ==0  => 0.1
lonilfiltered['trip_distance']= lonilfiltered['trip_distance'].replace({0:0.1})


<ipython-input-306-54d16e60fcbe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lonilfiltered['trip_distance']= lonilfiltered['trip_distance'].replace({0:0.1})


In [307]:
lonilfiltered


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,2020-06-01 00:31:23,2020-06-01 00:49:58,1.0,3.60,1.0,140,68,1.0,15.5,3.0,0.5,4.00,0.00,0.3,23.30,2.5,1115.0
1,2020-06-01 00:42:50,2020-06-01 01:04:33,1.0,5.60,1.0,79,226,1.0,19.5,3.0,0.5,2.00,0.00,0.3,25.30,2.5,1303.0
2,2020-06-01 00:39:51,2020-06-01 00:49:09,1.0,2.30,1.0,238,116,2.0,10.0,0.5,0.5,0.00,0.00,0.3,11.30,0.0,558.0
3,2020-06-01 00:56:13,2020-06-01 01:11:38,1.0,5.30,1.0,141,116,2.0,17.5,3.0,0.5,0.00,0.00,0.3,21.30,2.5,925.0
4,2020-06-01 00:16:41,2020-06-01 00:29:30,1.0,4.40,1.0,186,75,1.0,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499038,2020-06-30 23:07:00,2020-06-30 23:08:46,1.0,1.06,1.0,74,194,2.0,5.0,0.5,0.5,0.00,6.12,0.3,12.42,0.0,106.0
499039,2020-06-30 23:24:49,2020-06-30 23:32:00,1.0,1.52,1.0,138,70,1.0,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,431.0
499040,2020-06-30 23:57:38,2020-07-01 00:24:57,1.0,13.36,1.0,138,259,2.0,37.5,0.5,0.5,0.00,6.12,0.3,44.92,0.0,1639.0
499041,2020-06-30 23:08:03,2020-06-30 23:26:37,1.0,2.23,1.0,41,41,2.0,13.5,0.5,0.5,0.00,0.00,0.3,14.80,0.0,1114.0


In [308]:
sum(lonilfiltered['trip_distance']==0)

0

In [309]:
# Speed conditions
sc1= (lonilfiltered['trip_distance']/lonilfiltered['duration'])<=0.009 # < 34MPH
sc2= (lonilfiltered['trip_distance']/lonilfiltered['duration'])>0.00097 # >3.5MPH
lonilspfiltered= lonilfiltered[sc1 & sc2]

In [310]:
lonilspfiltered

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,2020-06-01 00:31:23,2020-06-01 00:49:58,1.0,3.60,1.0,140,68,1.0,15.5,3.0,0.5,4.00,0.00,0.3,23.30,2.5,1115.0
1,2020-06-01 00:42:50,2020-06-01 01:04:33,1.0,5.60,1.0,79,226,1.0,19.5,3.0,0.5,2.00,0.00,0.3,25.30,2.5,1303.0
2,2020-06-01 00:39:51,2020-06-01 00:49:09,1.0,2.30,1.0,238,116,2.0,10.0,0.5,0.5,0.00,0.00,0.3,11.30,0.0,558.0
3,2020-06-01 00:56:13,2020-06-01 01:11:38,1.0,5.30,1.0,141,116,2.0,17.5,3.0,0.5,0.00,0.00,0.3,21.30,2.5,925.0
4,2020-06-01 00:16:41,2020-06-01 00:29:30,1.0,4.40,1.0,186,75,1.0,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499036,2020-06-30 23:27:28,2020-06-30 23:39:29,1.0,2.50,1.0,186,148,1.0,11.0,3.0,0.5,1.00,0.00,0.3,15.80,2.5,721.0
499039,2020-06-30 23:24:49,2020-06-30 23:32:00,1.0,1.52,1.0,138,70,1.0,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,431.0
499040,2020-06-30 23:57:38,2020-07-01 00:24:57,1.0,13.36,1.0,138,259,2.0,37.5,0.5,0.5,0.00,6.12,0.3,44.92,0.0,1639.0
499041,2020-06-30 23:08:03,2020-06-30 23:26:37,1.0,2.23,1.0,41,41,2.0,13.5,0.5,0.5,0.00,0.00,0.3,14.80,0.0,1114.0


In [311]:
# FARE AMOUNT CONDITIONS

fc1= lonilspfiltered['fare_amount']<lonilspfiltered['trip_distance']*10
fc2= lonilspfiltered['fare_amount']>lonilspfiltered['trip_distance']*2.5
lonilspffiltered= lonilspfiltered[fc1 & fc2]

In [312]:
lonilspffiltered

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,2020-06-01 00:31:23,2020-06-01 00:49:58,1.0,3.60,1.0,140,68,1.0,15.5,3.0,0.5,4.00,0.00,0.3,23.30,2.5,1115.0
1,2020-06-01 00:42:50,2020-06-01 01:04:33,1.0,5.60,1.0,79,226,1.0,19.5,3.0,0.5,2.00,0.00,0.3,25.30,2.5,1303.0
2,2020-06-01 00:39:51,2020-06-01 00:49:09,1.0,2.30,1.0,238,116,2.0,10.0,0.5,0.5,0.00,0.00,0.3,11.30,0.0,558.0
3,2020-06-01 00:56:13,2020-06-01 01:11:38,1.0,5.30,1.0,141,116,2.0,17.5,3.0,0.5,0.00,0.00,0.3,21.30,2.5,925.0
4,2020-06-01 00:16:41,2020-06-01 00:29:30,1.0,4.40,1.0,186,75,1.0,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499036,2020-06-30 23:27:28,2020-06-30 23:39:29,1.0,2.50,1.0,186,148,1.0,11.0,3.0,0.5,1.00,0.00,0.3,15.80,2.5,721.0
499039,2020-06-30 23:24:49,2020-06-30 23:32:00,1.0,1.52,1.0,138,70,1.0,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,431.0
499040,2020-06-30 23:57:38,2020-07-01 00:24:57,1.0,13.36,1.0,138,259,2.0,37.5,0.5,0.5,0.00,6.12,0.3,44.92,0.0,1639.0
499041,2020-06-30 23:08:03,2020-06-30 23:26:37,1.0,2.23,1.0,41,41,2.0,13.5,0.5,0.5,0.00,0.00,0.3,14.80,0.0,1114.0


In [313]:
raw= lonilspffiltered

In [314]:
raw

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,2020-06-01 00:31:23,2020-06-01 00:49:58,1.0,3.60,1.0,140,68,1.0,15.5,3.0,0.5,4.00,0.00,0.3,23.30,2.5,1115.0
1,2020-06-01 00:42:50,2020-06-01 01:04:33,1.0,5.60,1.0,79,226,1.0,19.5,3.0,0.5,2.00,0.00,0.3,25.30,2.5,1303.0
2,2020-06-01 00:39:51,2020-06-01 00:49:09,1.0,2.30,1.0,238,116,2.0,10.0,0.5,0.5,0.00,0.00,0.3,11.30,0.0,558.0
3,2020-06-01 00:56:13,2020-06-01 01:11:38,1.0,5.30,1.0,141,116,2.0,17.5,3.0,0.5,0.00,0.00,0.3,21.30,2.5,925.0
4,2020-06-01 00:16:41,2020-06-01 00:29:30,1.0,4.40,1.0,186,75,1.0,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499036,2020-06-30 23:27:28,2020-06-30 23:39:29,1.0,2.50,1.0,186,148,1.0,11.0,3.0,0.5,1.00,0.00,0.3,15.80,2.5,721.0
499039,2020-06-30 23:24:49,2020-06-30 23:32:00,1.0,1.52,1.0,138,70,1.0,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,431.0
499040,2020-06-30 23:57:38,2020-07-01 00:24:57,1.0,13.36,1.0,138,259,2.0,37.5,0.5,0.5,0.00,6.12,0.3,44.92,0.0,1639.0
499041,2020-06-30 23:08:03,2020-06-30 23:26:37,1.0,2.23,1.0,41,41,2.0,13.5,0.5,0.5,0.00,0.00,0.3,14.80,0.0,1114.0


### AUGMENTED DATASET AND DOING THE TRANSOFMATION FOR PERIODICITY FEATURES

In [315]:
# AUGMENTED DATASET
raw['start_month']=raw['tpep_pickup_datetime'].dt.month_name()
raw['end_month']=raw['tpep_dropoff_datetime'].dt.month_name()
months =['January','February','March', 'April','May','June','July','August','September','October','November','December']
dmonths= dict(zip(months,np.arange(1,13,1)))
raw['start_month_num']=raw['start_month'].map(dmonths)
raw['end_month_num']=raw['end_month'].map(dmonths)

raw['start_day']=raw['tpep_pickup_datetime'].dt.dayofweek
raw['end_day']=raw['tpep_dropoff_datetime'].dt.dayofweek
dday= dict(zip(np.arange(0,7,1),[0,0,0,0,0,1,1]))
raw['weekend_starttrip']=raw['start_day'].map(dday)
raw['weekend_endtrip']=raw['end_day'].map(dday)

raw['start_hour']=raw['tpep_pickup_datetime'].dt.hour
raw['end_hour']=raw['tpep_dropoff_datetime'].dt.hour







<ipython-input-315-2c4379d363cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_month']=raw['tpep_pickup_datetime'].dt.month_name()
<ipython-input-315-2c4379d363cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['end_month']=raw['tpep_dropoff_datetime'].dt.month_name()
<ipython-input-315-2c4379d363cf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [316]:
#SIN COS TRANSFORM FOR MONTH DAY HOUR
raw['start_hour_sin']= np.sin(raw['start_hour']*(2.*np.pi/24)) # coordinate values on Oy
raw['start_hour_cos']= np.cos(raw['start_hour']*(2.*np.pi/24)) #coordinate values on Ox
raw['end_hour_sin']= np.sin(raw['end_hour']*(2.*np.pi/24)) # coordinate values on Oy
raw['end_hour_cos']= np.cos(raw['end_hour']*(2.*np.pi/24)) #coordinate values on Ox

raw['start_day_sin']= np.sin(raw['start_day']*(2.*np.pi/7))
raw['start_day_cos']= np.cos(raw['start_day']*(2.*np.pi/7))
raw['end_day_sin']= np.sin(raw['end_day']*(2.*np.pi/7))
raw['end_day_cos']= np.cos(raw['end_day']*(2.*np.pi/7))


raw['start_month_sin']= np.sin(raw['start_month_num']*(2.*np.pi/12))
raw['start_month_cos']= np.cos(raw['start_month_num']*(2.*np.pi/12))
raw['end_month_sin']= np.sin(raw['end_month_num']*(2.*np.pi/12))
raw['end_month_cos']= np.cos(raw['end_month_num']*(2.*np.pi/12))


<ipython-input-316-888254f2ea59>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_hour_sin']= np.sin(raw['start_hour']*(2.*np.pi/24)) # coordinate values on Oy
<ipython-input-316-888254f2ea59>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_hour_cos']= np.cos(raw['start_hour']*(2.*np.pi/24)) #coordinate values on Ox
<ipython-input-316-888254f2ea59>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

### HANDLING LONGITUDE LATITUDE FEATURES BASED ON LOCATIONID

In [317]:
def mapping():
    with open('D:/TOPIC_RELATED VOCABULARY IELTS/Finland/aalto/cemsschool/Other Courses/programming course for DS/project/taxi_zones.geojson') as f:
        gj = geojson.load(f)
    features = gj['features']
    return features

In [318]:
features=mapping()
coor0 = []
coor1= []


In [435]:
for i in np.arange(0,len(features)):
    if features[i]['geometry']['type']=='MultiPolygon':
        shape= features[i]['geometry']['coordinates'][0][0]
    else:   
        shape= features[i]['geometry']['coordinates'][0]
    centroid= list(Polygon(shape).centroid.coords)
    coor0.append(centroid[0][0])
    coor1.append(centroid[0][1])

In [439]:
features[131]

{"geometry": {"coordinates": [[[[1032791.0008, 181085.006298], [1032839.294394, 181083.884491], [1033003.132717, 181084.130279], [1033338.400922, 181070.396701], [1034765.976964, 180861.647125], [1035584.328147, 180687.264319], [1035734.554274, 180702.677708], [1035884.337661, 180723.333778], [1036033.398881, 180749.209701], [1036181.469659, 180780.259352], [1036328.276753, 180816.424197], [1036472.394098, 180857.427903], [1036638.902776, 180946.295983], [1036792.816584, 181057.158381], [1036942.960692, 181170.934776], [1037111.744473, 181263.052845], [1037281.698003, 181355.80967], [1037391.368632, 181415.665126], [1037572.256392, 181462.588716], [1037613.75524, 181473.353919], [1038030.809312, 181581.540512], [1038147.284559, 181599.253925], [1038536.513714, 181658.44646], [1038687.75066, 181673.15792], [1038904.08846, 181694.201496], [1039138.930794, 181706.878249], [1039171.276767, 181706.248902], [1039211.614132, 181706.665079], [1039235.521055, 181707.274397], [1039532.794717, 18

In [320]:
start_pos= pd.DataFrame([coor0,coor1]).T

In [321]:
start_pos.rename(columns={0:'coor0',1:'coor1'})

,coor0,coor1
0,9.359968e+05,191376.749531
1,1.032418e+06,166735.079475
2,1.026453e+06,254265.478658
3,9.906340e+05,202959.782391
4,9.318714e+05,140681.351376
...,...,...
258,1.025106e+06,266453.414552
259,1.010212e+06,210434.891894
260,9.806395e+05,197635.173703
261,9.990266e+05,221945.372994


In [322]:
###LON LAT OF SOME SPECIFIC PLACES
JFKairport= (-73.780968, 40.641766)

NWairport=(-74.172363, 40.735657)

#MIDTOWN MANHATTAN
#UES the mark hotel 40.77541739176158, -73.96342029243233
Mark= (-73.96342029243233,40.77541739176158)
Timesquare = (-73.98604331729206,40.75837054036599)
Grand =( -73.97711770261203,40.75284316174533)
Penn= (-73.99368229963214, 40.75069857282476)
Lincoln= (-73.9839037407981, 40.773072825611635)

#West side of manhattan Bloomingdale flor de mayo 40.797800366747595, -73.9700034852982
Flor = (-73.9700034852982,40.797800366747595)

# Lower manhattan one world trade center 40.71315120718066, -74.01335094122703
Wtc=  (-74.01335094122703,40.71315120718066)

#Upper manhattan bank of america 40.82320010657351, -73.9420433120473
Boa = (-73.9420433120473, 40.82320010657351)

L= [JFKairport, NWairport, Mark,Timesquare,Grand,Penn,Lincoln,Flor,Wtc,Boa]

In [443]:
#Converting decimal lon lat to radians for each specific places mentioned above
def detora(List):
    res=[]
    for i in List:
        listc= list(map(radians,i))
        res.append(listc)
    return res


In [ ]:
res= detora(L)

In [73]:
# convert decimal lon lat to radians for PUlon and PUlat 
puloc= pd.Series(map(radians,raw['pickup_longitude']))
pulac= pd.Series(map(radians,raw['pickup_latitude']))

KeyError: 'pickup_longitude'

In [32]:
#Compute the Harvesine distance between the datapoints and each specific place listed above
def haversineindf(df):
    for i in np.arange(0,10):
        dlon= puloc-res[i][0]
        dlat= pulac-res[i][1]
        a= np.sin(dlat/2)**2 + np.cos(pulac) * cos(res[i][1]) * np.sin(dlon/2)**2
        c= 2*np.arcsin(np.sqrt(a))
        s= pd.Series(c*3956).rename(i)
        df= pd.concat((df,s),axis=1)
    return df

In [49]:
puloc

NameError: name 'puloc' is not defined

In [50]:
s1= pd.Series(np.zeros(len(puloc)))
df= haversineindf(s1)

NameError: name 'puloc' is not defined

In [51]:
df=df.iloc[:,1::]

In [52]:
df.head()

,Borough,Zone,service_zone,Unnamed: 4,Unnamed: 5
1,Queens,Jamaica Bay,Boro Zone,NaN,NaN
6,Queens,Astoria,Boro Zone,NaN,NaN
7,Queens,Astoria Park,Boro Zone,NaN,NaN
8,Queens,Auburndale,Boro Zone,NaN,NaN
9,Queens,Baisley Park,Boro Zone,NaN,NaN


In [59]:
dfscaled=dfscaled.rename(columns=dict(zip(np.arange(0,10),['tojfk','tonw', 'tomark','totimes','togrand','topenn','tolincoln','toflor','towtc','toboa'])))

NameError: name 'dfscaled' is not defined

In [60]:
#the haversine distance computed represented by a dataframe
dfscaled

NameError: name 'dfscaled' is not defined

In [61]:
Xfull= pd.concat((raw,dfscaled),axis=1)

NameError: name 'dfscaled' is not defined

#### Approach 2.2: some specific places in area having the most dropoff
(this is the initial approach using the original coordinates. You could drop this part when  the above part after being transformed to lon lat works properly. Otherwise, leave this part untouched since it could function so that you could proceed the codes in the below sections)

In [323]:
#distance to JFK
JFKdis= np.sqrt((start_pos[0]-1043400.11687483)**2+(start_pos[1]-175000.69898434)**2)
UESdis= np.sqrt((start_pos[0]-996250.38676579)**2+(start_pos[1]- 2236700.75357605)**2)
NAdis=np.sqrt((start_pos[0]-935900.82101626)**2+(start_pos[1]- 191300.74953085)**2)

Tridis= np.sqrt((start_pos[0]-982100.70138546)**2+(start_pos[1]-200700.54042726)**2)
Timdis= np.sqrt((start_pos[0]-988600.06004673)**2+(start_pos[1]-216000.13976843)**2)
UWdis= np.sqrt((start_pos[0]-993100.37686082)**2+(start_pos[1]-221000.64462577)**2)



In [324]:
dJ= dict(zip(np.arange(1,264),JFKdis))
dU= dict(zip(np.arange(1,264),UESdis))
dN= dict(zip(np.arange(1,264),NAdis))
dTr= dict(zip(np.arange(1,264),Tridis))
dTi= dict(zip(np.arange(1,264),Timdis))
dU= dict(zip(np.arange(1,264),UWdis))

In [335]:
dJSeries= raw['PULocationID'].map(dJ)
dUSeries= raw['PULocationID'].map(dU)
dNSeries=raw['PULocationID'].map(dN)
dJSeries=dJSeries.rename('JFKdis')
dUeSeries=dUSeries.rename('UESdis')
dNSeries= dNSeries.rename('NAdis')

dTrSeries= raw['PULocationID'].map(dTr)
dTiSeries= raw['PULocationID'].map(dTi)
dUSeries=raw['PULocationID'].map(dU)
dTrSeries=dTrSeries.rename('Tridis')
dTiSeries=dTiSeries.rename('Timdis')
dUwSeries=dUSeries.rename('UWdis')


In [336]:
X= pd.concat((dJSeries,dUeSeries,dNSeries, dTrSeries,dTiSeries,dUwSeries),axis=1)

In [337]:
X.describe()

,JFKdis,UESdis,NAdis,Tridis,Timdis,UWdis
count,433560.000000,433560.000000,433560.000000,433560.000000,433560.000000,433560.000000
mean,66639.624024,10023.488977,63015.105890,20891.208142,10001.659737,10023.488977
std,8176.723324,8685.340309,9497.763333,10465.076672,8925.098815,8685.340309
min,107.647573,1821.947475,122.441823,87.321246,102.883429,1821.947475
25%,64623.264905,4875.917789,56398.949394,12921.202432,5189.333859,4875.917789
50%,66717.411515,7250.136981,61223.978315,21685.185107,7345.020492,7250.136981
75%,70019.690273,12643.589157,68372.091705,26880.281210,11933.112827,12643.589157
max,131707.849207,116517.763142,129952.547662,93940.792646,109798.304133,116517.763142


In [338]:
Xfull=pd.concat((raw,X),axis=1)

### TRIMMING IRRELAVANT PARTS

In [339]:
Xfull.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration', 'start_month', 'end_month',
       'start_month_num', 'end_month_num', 'start_day', 'end_day',
       'weekend_starttrip', 'weekend_endtrip', 'start_hour', 'end_hour',
       'start_hour_sin', 'start_hour_cos', 'end_hour_sin', 'end_hour_cos',
       'start_day_sin', 'start_day_cos', 'end_day_sin', 'end_day_cos',
       'start_month_sin', 'start_month_cos', 'end_month_sin', 'end_month_cos',
       'JFKdis', 'UESdis', 'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [340]:
Xfull.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime','start_month', 'end_month',
       'start_month_num', 'end_month_num', 'start_day', 'end_day','start_hour', 'end_hour', 'PULocationID', 'DOLocationID'],axis=1, inplace=True)

In [341]:
Xuse=Xfull.loc[:,~Xfull.columns.duplicated()]

In [481]:
Xuse.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

### SCALING DATA

In [261]:
rs=RobustScaler()

In [343]:
rs.fit(Xuse)

RobustScaler()

In [451]:
Xscaled= pd.DataFrame(rs.transform(Xuse),index=Xuse.index,columns=Xuse.columns)


### QUEENS TASK 3

#### 3a Predicting trip distance for Queens

In [464]:
np.random.seed(0)

In [465]:

Xscaled.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [466]:
ya=Xscaled['trip_distance']

In [467]:
Xa= Xscaled.drop(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration','weekend_endtrip','end_hour_sin', 'end_hour_cos','end_day_sin','end_day_cos','end_month_sin','end_month_cos'],axis=1)


In [468]:
Xa

,weekend_starttrip,start_hour_sin,start_hour_cos,start_day_sin,start_day_cos,start_month_sin,start_month_cos,JFKdis,UESdis,NAdis,Tridis,Timdis,UWdis
0,0.0,0.366025,1.732051,0.000000,0.801938,0.0,0.0,-0.590739,-0.334651,0.444875,0.081314,0.166719,-0.334651
1,0.0,0.366025,1.732051,0.000000,0.801938,0.0,0.0,-0.768108,1.300745,-0.615353,-1.046826,0.636944,1.300745
2,0.0,0.366025,1.732051,0.000000,0.801938,0.0,0.0,1.317738,-0.050392,0.452513,0.500803,0.708616,-0.050392
3,0.0,0.366025,1.732051,0.000000,0.801938,0.0,0.0,-0.338829,-0.511680,0.359869,0.051038,0.000000,-0.511680
4,0.0,0.366025,1.732051,0.000000,0.801938,0.0,0.0,0.235356,0.518021,-0.560204,-0.690495,-0.404957,0.518021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499036,0.0,0.176557,1.692705,0.643104,0.554958,0.0,0.0,0.235356,0.518021,-0.560204,-0.690495,-0.404957,0.518021
499039,0.0,0.176557,1.692705,0.643104,0.554958,0.0,0.0,-2.675402,2.446206,2.296198,1.499901,3.540645,2.446206
499040,0.0,0.176557,1.692705,0.643104,0.554958,0.0,0.0,-2.675402,2.446206,2.296198,1.499901,3.540645,2.446206
499041,0.0,0.176557,1.692705,0.643104,0.554958,0.0,0.0,1.217398,0.641420,1.084023,0.973513,1.684289,0.641420


In [469]:
Xtrain,Xtest,ytrain,ytest= train_test_split(Xa,ya,test_size=0.2)

In [470]:
# Which model, which hyperparameters to use? I do Cross-validation to check
modelhya= SGDRegressor(random_state=True)
hyperparameters= {'loss':['squared_loss','huber','squared_epsilon_insensitive'], 'alpha': np.linspace(0,0.001, 5)} 
#,'squared_epsilon_insensitive'

In [471]:
grid= GridSearchCV(modelhya, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [472]:
grid.fit(Xtrain,ytrain)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=True), n_jobs=-1,
             param_grid={'alpha': array([0.     , 0.00025, 0.0005 , 0.00075, 0.001  ]),
                         'loss': ['squared_loss', 'huber',
                                  'squared_epsilon_insensitive']},
             scoring='neg_root_mean_squared_error')

In [474]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','huber0','squared_ep0','squared.25','huber.25','squared_ep.25','squared.50','huber50','squared_ep.50','squared.75','huber75','squared_ep.75','squared1','huber1','squared_ep1'])
# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [475]:
comparison #=>squared1 best => alpha = 1, loss= squared is the best( see the results on real queens dataset). USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,1.232111
huber0,1.277951
squared_ep0,1.233513
squared.25,1.232109
huber.25,1.278113
squared_ep.25,1.233513
squared.50,1.232107
huber50,1.278258
squared_ep.50,1.233513
squared.75,1.232105


In [356]:
modela =SGDRegressor(alpha= 1, random_state=True)

In [357]:
modela.fit(Xtrain,ytrain)

SGDRegressor(alpha=1, random_state=True)

In [358]:
modela.score(Xtrain,ytrain)

0.1970052542776891

In [359]:
modela.score(Xtest,ytest)

0.1927796251879833

#### 3b Predicting fare_amount (parsimonious model)

In [364]:
yb= Xscaled['fare_amount']

In [365]:
Xscaled.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [367]:
Xb =Xscaled.drop(['payment_type','fare_amount',
                      'extra',
                     'mta_tax',            'tip_amount',
                'tolls_amount', 'improvement_surcharge',
                'total_amount',  'congestion_surcharge',
                ],axis=1)

In [368]:
Xb.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'duration',
       'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [371]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xb,yb,test_size=0.2)

In [370]:
# Which model, which hyperparameters to use? I do the Cross-validation to check
modelhyb= SGDRegressor(random_state=True)
hyperparameters= {'loss':['squared_loss','huber','squared_epsilon_insensitive'], 'alpha': np.linspace(0,0.001, 5)} 
#,'squared_epsilon_insensitive'

In [374]:
grid= GridSearchCV(modelhyb, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [375]:
grid.fit(Xtrain,ytrain)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=True), n_jobs=-1,
             param_grid={'alpha': array([0.     , 0.00025, 0.0005 , 0.00075, 0.001  ]),
                         'loss': ['squared_loss',
                                  'squared_epsilon_insensitive']},
             scoring='neg_root_mean_squared_error')

In [376]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','huber0','squared_ep0','squared.25','huber.25','squared_ep.25','squared.50','huber50','squared_ep.50','squared.75','huber75','squared_ep.75','squared1','huber1','squared_ep1'])

# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [378]:
comparison #=>squared1 best => alpha = .25, loss= squared_loss is the best. USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,1.344923e+09
squared_ep0,5.334084e+09
squared.25,1.251926e+09
squared_ep.25,5.320682e+09
squared.50,1.291309e+09
squared_ep.50,5.305431e+09
squared.75,1.414075e+09
squared_ep.75,5.290541e+09
squared1,1.521205e+09
squared_ep1,5.277427e+09


In [379]:
modelb =SGDRegressor(alpha= .25, random_state=True)

In [380]:
modelb.fit(Xtrain,ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [381]:
modelb.score(Xtrain,ytrain)

0.9802870257914063

In [382]:
modelb.score(Xtest,ytest)

0.9846889841694365

In [384]:
Xscaled.corr()

,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,...,start_month_sin,start_month_cos,end_month_sin,end_month_cos,JFKdis,UESdis,NAdis,Tridis,Timdis,UWdis
passenger_count,1.000000,0.010001,-0.002966,-0.002624,0.010392,-0.163364,0.015422,0.002412,0.002688,0.004451,...,-0.001705,-0.001082,-0.000476,-0.000941,-0.002091,0.009865,0.007464,0.007923,0.014112,0.009865
trip_distance,0.010001,1.000000,0.055931,-0.012342,0.977954,-0.003467,-0.039438,0.346553,0.442942,-0.021732,...,0.000687,-0.000941,-0.014712,0.003142,-0.372150,0.409168,0.257738,0.217359,0.390136,0.409168
RatecodeID,-0.002966,0.055931,1.000000,0.003145,0.057443,-0.003604,-0.060524,0.024021,0.052469,-0.002013,...,-0.000057,-0.000057,-0.000764,0.000166,-0.044354,0.040811,0.030229,0.028145,0.041166,0.040811
payment_type,-0.002624,-0.012342,0.003145,1.000000,-0.007369,-0.033047,-0.001694,-0.542634,-0.014110,-0.011875,...,0.003699,0.003451,0.000558,0.003330,0.000398,0.050943,0.064753,0.068134,0.069915,0.050943
fare_amount,0.010392,0.977954,0.057443,-0.007369,1.000000,0.001258,-0.055967,0.349590,0.420287,-0.024548,...,0.000409,-0.000888,-0.012579,0.002624,-0.334942,0.379785,0.238026,0.202517,0.364048,0.379785
extra,-0.163364,-0.003467,-0.003604,-0.033047,0.001258,1.000000,-0.034379,0.036549,-0.012336,-0.000496,...,-0.002118,-0.002480,-0.001595,-0.001975,0.011319,-0.080023,-0.120102,-0.125098,-0.126779,-0.080023
mta_tax,0.015422,-0.039438,-0.060524,-0.001694,-0.055967,-0.034379,1.000000,-0.018659,-0.073116,0.008357,...,0.000146,0.000146,-0.000494,0.000280,0.004193,-0.007481,-0.006724,-0.007614,-0.009296,-0.007481
tip_amount,0.002412,0.346553,0.024021,-0.542634,0.349590,0.036549,-0.018659,1.000000,0.191610,0.006443,...,-0.002149,-0.002084,-0.002304,-0.001411,-0.130923,0.103609,0.039656,0.020798,0.083747,0.103609
tolls_amount,0.002688,0.442942,0.052469,-0.014110,0.420287,-0.012336,-0.073116,0.191610,1.000000,-0.005888,...,-0.000382,-0.000383,-0.009279,0.002303,-0.175615,0.198026,0.189170,0.155462,0.211100,0.198026
improvement_surcharge,0.004451,-0.021732,-0.002013,-0.011875,-0.024548,-0.000496,0.008357,0.006443,-0.005888,1.000000,...,0.000033,0.000034,-0.000113,0.000064,0.004717,-0.006326,-0.003627,-0.003017,-0.006231,-0.006326


##### 8 strongest features trip_distance and duration + 6 distance features

In [413]:
modelb8= SGDRegressor(alpha=.25,random_state=True)

In [415]:
modelb8.fit(Xtrain[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [416]:
modelb8.score(Xtrain[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

0.9802298269203806

In [432]:
modelb8.score(Xtest[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytest)

NameError: name 'modelb8' is not defined

##### What about 6 features which are not really strongly linearly correalated to the fare_amount

In [426]:
modelb6= SGDRegressor(alpha=.25,random_state=True)

In [427]:
modelb6.fit(Xtrain[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [428]:
modelb6.score(Xtrain[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

0.1636583740198263

In [429]:
modelb6.score(Xtest[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytest)

0.1674989633191657

##### 2 strongest features trip_distance and duration

In [409]:
modelb2= SGDRegressor(alpha=.25,random_state=True)

In [410]:
modelb2.fit(Xtrain[['trip_distance','duration']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [411]:
modelb2.score(Xtrain[['trip_distance','duration']],ytrain)

0.9811036953226984

In [412]:
modelb2.score(Xtest[['trip_distance','duration']],ytest)

0.9850873243666176

##### Could the model be more parsimonious? 1 feature only. Either trip_distance or duration

In [418]:
modelb1= SGDRegressor(alpha=.25,random_state=True)

In [419]:
modelb1.fit(Xtrain[['trip_distance']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [420]:
modelb1.score(Xtrain[['trip_distance']],ytrain)

0.9418774728552727

In [421]:
modelb1.score(Xtest[['trip_distance']],ytest)

0.9414299598377417

In [422]:
modelbdu= SGDRegressor(alpha=.25,random_state=True)

In [423]:
modelbdu.fit(Xtrain[['duration']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [441]:
modelbdu.score(Xtrain[['duration']],ytrain)


0.8239626235232147

In [442]:
modelbdu.score(Xtest[['duration']],ytest)

0.8317269995519062

####### => conclusion: 2 features should be best both in terms of the performance of the model and the number of parameters

#### 3c Predict payment type

In [478]:
Xscaled.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [489]:
yc= Xscaled['payment_type']

In [490]:
Xc=Xscaled.drop(['payment_type'],axis=1)

In [507]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xc,yc,test_size=0.2)

In [503]:
# Which model, which hyperparameters to use? I do the Cross-validation to check
modelhyc= SGDClassifier(random_state=True)
hyperparameters= {'loss':['log','hinge','squared_hinge'], 'alpha': np.linspace(0.0001,0.001, 5)} 


In [508]:
ytrain

244959    0.0
336672    1.0
145283    0.0
258496    1.0
229141    1.0
         ... 
227593    0.0
306370    0.0
99052     0.0
279462    0.0
463010    0.0
Name: payment_type, Length: 346848, dtype: float64

In [509]:
grid= GridSearchCV(modelhyc, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [510]:
grid.fit(Xtrain,ytrain)

KeyboardInterrupt: 

In [376]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','squared_ep0','squared.25','squared_ep.25','squared.50','squared_ep.50','squared.75','squared_ep.75','squared1','squared_ep1'])
# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [378]:
comparison #=>squared1 best => alpha = .25, loss= squared_loss is the best. USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,1.344923e+09
squared_ep0,5.334084e+09
squared.25,1.251926e+09
squared_ep.25,5.320682e+09
squared.50,1.291309e+09
squared_ep.50,5.305431e+09
squared.75,1.414075e+09
squared_ep.75,5.290541e+09
squared1,1.521205e+09
squared_ep1,5.277427e+09


In [512]:
modelc= SGDClassifier(alpha=.25,random_state=True)

In [513]:
modelc.fit(Xtrain,ytrain)

SGDClassifier(alpha=0.25, random_state=True)

In [514]:
modelc.score(Xtrain,ytrain)


0.7397851508441738

In [516]:
modelc.score(Xtest,ytest)

0.7369337577267275

### ENTIRE DATASET TASK 3

#### 3a Predicting the trip_distance

In [356]:
modelae =SGDRegressor(alpha= 1, random_state=True) #best hyperparameter found in Queens would be used 

In [357]:
# Place it within loop through chunks
modelae.partial_fit(Xtrain,ytrain)

SGDRegressor(alpha=1, random_state=True)

In [358]:
# Place it out of the loop
modelae.score(Xtrain,ytrain)

0.1970052542776891

In [359]:
# place it out of the loop
modelae.score(Xtest,ytest)

0.1927796251879833

#### 3b Predicting the fare_amount

In [379]:
modelb =SGDRegressor(alpha= .25, random_state=True)

In [380]:
# Place it within loop through chunks
modelb.partial_fit(Xtrain,ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [381]:
#PLace it out of the loop
modelb.score(Xtrain,ytrain)

0.9802870257914063

In [382]:
#place it out of the loop
modelb.score(Xtest,ytest)

0.9846889841694365

In [384]:
Xscaled.corr()

,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,...,start_month_sin,start_month_cos,end_month_sin,end_month_cos,JFKdis,UESdis,NAdis,Tridis,Timdis,UWdis
passenger_count,1.000000,0.010001,-0.002966,-0.002624,0.010392,-0.163364,0.015422,0.002412,0.002688,0.004451,...,-0.001705,-0.001082,-0.000476,-0.000941,-0.002091,0.009865,0.007464,0.007923,0.014112,0.009865
trip_distance,0.010001,1.000000,0.055931,-0.012342,0.977954,-0.003467,-0.039438,0.346553,0.442942,-0.021732,...,0.000687,-0.000941,-0.014712,0.003142,-0.372150,0.409168,0.257738,0.217359,0.390136,0.409168
RatecodeID,-0.002966,0.055931,1.000000,0.003145,0.057443,-0.003604,-0.060524,0.024021,0.052469,-0.002013,...,-0.000057,-0.000057,-0.000764,0.000166,-0.044354,0.040811,0.030229,0.028145,0.041166,0.040811
payment_type,-0.002624,-0.012342,0.003145,1.000000,-0.007369,-0.033047,-0.001694,-0.542634,-0.014110,-0.011875,...,0.003699,0.003451,0.000558,0.003330,0.000398,0.050943,0.064753,0.068134,0.069915,0.050943
fare_amount,0.010392,0.977954,0.057443,-0.007369,1.000000,0.001258,-0.055967,0.349590,0.420287,-0.024548,...,0.000409,-0.000888,-0.012579,0.002624,-0.334942,0.379785,0.238026,0.202517,0.364048,0.379785
extra,-0.163364,-0.003467,-0.003604,-0.033047,0.001258,1.000000,-0.034379,0.036549,-0.012336,-0.000496,...,-0.002118,-0.002480,-0.001595,-0.001975,0.011319,-0.080023,-0.120102,-0.125098,-0.126779,-0.080023
mta_tax,0.015422,-0.039438,-0.060524,-0.001694,-0.055967,-0.034379,1.000000,-0.018659,-0.073116,0.008357,...,0.000146,0.000146,-0.000494,0.000280,0.004193,-0.007481,-0.006724,-0.007614,-0.009296,-0.007481
tip_amount,0.002412,0.346553,0.024021,-0.542634,0.349590,0.036549,-0.018659,1.000000,0.191610,0.006443,...,-0.002149,-0.002084,-0.002304,-0.001411,-0.130923,0.103609,0.039656,0.020798,0.083747,0.103609
tolls_amount,0.002688,0.442942,0.052469,-0.014110,0.420287,-0.012336,-0.073116,0.191610,1.000000,-0.005888,...,-0.000382,-0.000383,-0.009279,0.002303,-0.175615,0.198026,0.189170,0.155462,0.211100,0.198026
improvement_surcharge,0.004451,-0.021732,-0.002013,-0.011875,-0.024548,-0.000496,0.008357,0.006443,-0.005888,1.000000,...,0.000033,0.000034,-0.000113,0.000064,0.004717,-0.006326,-0.003627,-0.003017,-0.006231,-0.006326


##### 8 strongest features trip_distance and duration + 6 distance features (if could, create a table or a file incorporating only these 8 features from the outset. Doing that would help reduce the time it takes to go about the codes)

In [413]:
modelb8= SGDRegressor(alpha=.25,random_state=True)

In [415]:
# within loop through chunks
modelb8.partial_fit(Xtrain[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [416]:
# out of loop
modelb8.score(Xtrain[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

0.9802298269203806

In [432]:
# out of loop
modelb8.score(Xtest[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytest)

NameError: name 'modelb8' is not defined

##### What about 6 features which are not really strongly linearly correalated to the fare_amount (same idea as above. Create a table or a file of only these 6 features before running model would be helpful to cut the waiting time drastically)

In [426]:
modelb6= SGDRegressor(alpha=.25,random_state=True)

In [427]:
# within loop through chunks
modelb6.partial_fit(Xtrain[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [428]:
#out of loop
modelb6.score(Xtrain[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

0.1636583740198263

In [429]:
# out of loop
modelb6.score(Xtest[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytest)

0.1674989633191657

##### 2 strongest features trip_distance and duration(same idea as above)

In [409]:
modelb2= SGDRegressor(alpha=.25,random_state=True)

In [410]:
#within loop
modelb2.partial_fit(Xtrain[['trip_distance','duration']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [411]:
# out of loop
modelb2.score(Xtrain[['trip_distance','duration']],ytrain)

0.9811036953226984

In [412]:
# out of loop
modelb2.score(Xtest[['trip_distance','duration']],ytest)

0.9850873243666176

##### Could the model be more parsimonious? 1 feature only. Either trip_distance or duration

In [418]:
modelb1= SGDRegressor(alpha=.25,random_state=True)

In [419]:
#within loop
modelb1.partial_fit(Xtrain[['trip_distance']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [420]:
# out of loop
modelb1.score(Xtrain[['trip_distance']],ytrain)

0.9418774728552727

In [421]:
# out of loop
modelb1.score(Xtest[['trip_distance']],ytest)

0.9414299598377417

In [422]:
modelbdu= SGDRegressor(alpha=.25,random_state=True)

In [423]:
#within loop
modelbdu.partial_fit(Xtrain[['duration']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [424]:
#out of loop
modelbdu.score(Xtrain[['duration']],ytrain)

0.8239626235232147

In [425]:
#out of loop
modelbdu.score(Xtest[['duration']],ytest)

0.8317269995519062

####### => conclusion: 2 features should be best both in terms of the performance of the model and the number of parameters